# Machine Learning for Neuroimaging 

## Hands on Statistics & ML (Python) - 10/19/2023
### Part 2: Classifier Training and Evaluation with PyTorch

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import metrics # Metrics for classification
from statsmodels.stats import contingency_tables # Contringency tables
import torch # PyTorch
from torch import nn # Modules and layers
from torch.utils.data import Dataset # PyTorch dataset
from torch.utils.data import DataLoader # PyTorch Dataloader

## Data Loading

In [3]:
# Read csv dataset
dataframe = pd.read_csv('data/autism_screening_preprocessed.csv')
dataframe.describe()

# Notice that we have the residualized "_prime" features!

,Unnamed: 0,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,...,A1_Score_prime,A2_Score_prime,A3_Score_prime,A4_Score_prime,A5_Score_prime,A6_Score_prime,A7_Score_prime,A8_Score_prime,A9_Score_prime,A10_Score_prime
count,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,...,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000
mean,351.926031,0.721195,0.453770,0.458037,0.496444,0.499289,0.284495,0.418208,0.650071,0.324324,...,0.755879,0.477765,0.456637,0.524960,0.520699,0.323009,0.386890,0.620102,0.319345,0.603399
std,203.201765,0.448731,0.498213,0.498591,0.500343,0.500355,0.451495,0.493616,0.477287,0.468455,...,0.447490,0.495977,0.496188,0.496722,0.499915,0.448073,0.492705,0.475392,0.464583,0.493442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.004978,-0.132533,-0.137897,-0.140061,-0.012749,-0.131993,-0.061520,-0.095625,-0.197031,-0.072568
25%,176.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.068227,0.029974,0.008382,0.044820,0.033673,0.021552,-0.059586,-0.000606,-0.002144,0.050891
50%,352.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.000915,0.084272,0.051417,0.120803,0.045899,0.082878,0.000272,0.926315,0.041218,0.986752
75%,527.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.066044,1.017473,1.000976,1.033614,1.016575,0.963454,0.940326,0.982732,0.939252,1.025794
max,703.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.069100,1.096773,1.061505,1.120803,1.045899,1.124376,1.000536,1.090287,1.075534,1.082833


## Autism Classification 

Train a model $f$ that given an input $x$ predicts a classification label $y$

In this problem $x$ denotes the input features of the questionnaire and $y$ is the subject classification to control and ASD based on their responses

First, prepare your data for PyTorch!

In [4]:
# Preparing data for classification

# The features we want to use to classify Control vs ASD are the responses to the autism questionnaire
feature_cols = ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 
                               'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']
# After the residualization!
feature_cols = [col+'_prime' for col in feature_cols]

target_col = 'Class/ASD'

# Transform target column into a binary label
labels = {"NO": 0, "YES": 1}
dataframe = dataframe.replace({target_col: labels})

In [5]:
# Divide the dataset into training and testing
train_ds = dataframe.sample(frac = 0.75)
test_ds = dataframe.drop(train_ds.index)

# Sometimes, you need to pay attention to certain examples remaining together! 
# For instance, scans from the same subject at different timepoints, or siblings/members of the same family

print(f"The initial dataframe had {len(dataframe)} individuals. Our train dataset contains {len(train_ds)} and our test dataset, {len(test_ds)}.")

# If some classes are under-represented (often the case with multi-class problems), the subsets must 
# be stratified. I.e., the proportion of classes is maintained

The initial dataframe had 703 individuals. Our train dataset contains 527 and our test dataset, 176.


## Datasets and Dataloaders in PyTorch

In [5]:
# Build a PandasDataset in the PyTorch format
class PandasDataset(Dataset):
    def __init__(self, df, feature_cols, target_col, y_dtype=torch.int64):
        self.X = torch.tensor(df[feature_cols].values, dtype=torch.float32)
        self.y = torch.tensor(df[target_col].values, dtype=y_dtype)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [6]:
# From that class, prepare the datasets in the appropiate format
train_pt_ds = PandasDataset(train_ds, feature_cols=feature_cols, target_col=target_col)
test_pt_ds = PandasDataset(test_ds, feature_cols=feature_cols, target_col=target_col)

In [7]:
# Now build the dataloaders from those datasets
batch_size = 5  # Number of samples on each iteration, select max. possible for the available memory

# Create dataloaders
train_dl = DataLoader(train_pt_ds, batch_size=batch_size, shuffle=True) # Shuffle the train dataset!
test_dl = DataLoader(test_pt_ds, batch_size=batch_size, shuffle=False)

for X, y in train_dl:
    print(f"Example X: {X.numpy()}")
    print(f"Shape of X: {X.shape}")
    print(f"Example y: {y}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Example X: [[ 1.000697    0.01330662  1.0161082   1.017332    1.0012801   0.01325238
   1.0001404   0.9899909   1.0197824   1.0072861 ]
 [ 0.9991692   0.9841387   0.98079944  0.9793406   0.99847424  0.9842034
   0.99983263  1.0119307   0.97641975  0.9913152 ]
 [ 0.06058744 -0.06573388 -0.13789669 -0.09086361  0.03026628 -0.03746829
  -0.06116859  1.0266112  -0.17559499 -0.00614747]
 [ 1.067572    0.06760496  1.0235145   1.0828112   1.0430927   1.0953271
   0.9402383   0.92631495  1.0226341   0.06686208]
 [ 1.0013518   0.02580713  0.03124042  1.033614    0.00248251  0.02570195
   1.0002723  -0.01941189  0.03836637  1.0141307 ]]
Shape of X: torch.Size([5, 10])
Example y: tensor([1, 1, 0, 1, 0])
Shape of y: torch.Size([5]) torch.int64


## PyTorch Models: building a linear classifier

In [8]:
# Build a classifier in PyTorch
class FullyConnectedClassifier(nn.Module):
    def __init__(self, input_size=None, nr_classes=2, hidden_layer_sizes=[16,8], save_path="models"):
        super().__init__()
        self.name = self.__class__.__name__ + '-'.join([str(n) for n in hidden_layer_sizes])
        self.save_path = save_path
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        # Define the architecture of the model
        layers = []
        nr_neurons = [input_size] + hidden_layer_sizes
        for ix in range(len(nr_neurons)-1):
            # A linear, fully-connected layer
            layers.append(nn.Linear(nr_neurons[ix], nr_neurons[ix+1]))
            # A ReLU activation function
            layers.append(nn.ReLU())
        # Final layer, in this case for binary (nr_classes==2) classification
        layers.append(nn.Linear(nr_neurons[-1], nr_classes))
        # Finally, we place them one after the other
        self.layers = nn.Sequential(*layers)
        
        # We'll sometimes need to flatten the inputs (not necessary in this case)
        self.flatten = nn.Flatten()
        # The softmax function ensures we have one output per class, and these add up to 1
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        '''Forward pass'''
        x = self.flatten(x)
        logits = self.layers(x)
        return logits
    
    def predict(self, X):
        '''Make a prediction based on a given input'''
        self.eval()
        with torch.no_grad():
            pred = self(X)
            return int(pred.argmax().detach())     
        
    def save(self, state_name='last', verbose=False):
        '''Saves a model state in the defined path, with the model name'''
        model_state_name = self.name+'_'+state_name+'.pth'
        torch.save(self.state_dict(), os.path.join(self.save_path, model_state_name))
        if verbose:
            print("Saved PyTorch model state {} in {}".format(model_state_name, self.save_path))
            
    def restore(self, state_name):
        '''Restores a model state for the given state name'''
        model_state_name = self.name+'_'+state_name+'.pth'
        self.load_state_dict(torch.load(os.path.join(self.save_path, model_state_name)))

In [9]:
# Print out model summary
model = FullyConnectedClassifier(input_size=len(feature_cols), hidden_layer_sizes=[2, 2])
print(model)

FullyConnectedClassifier(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=2, bias=True)
    (3): ReLU()
    (4): Linear(in_features=2, out_features=2, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax): Softmax(dim=1)
)


In [10]:
# Methods to train and evaluate model
def train_model(model, train_dl, optimizer, loss_f, nr_epochs, print_loss_every=10):
    for t in range(nr_epochs):
        model.train()
        nr_batches = len(train_dl)
        total_loss = 0
        optimizer.zero_grad()
        for _, (X, y) in enumerate(train_dl):
            # Backpropagation step
            pred = model(X)
            loss = loss_f(pred, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()
        if t % print_loss_every == 0:
            print(f"Epoch {t} loss {total_loss / nr_batches}")
    model.save()
    print("Finished training!")
            
def eval_model(model, dl):
    model.eval() # This is important for certain stochastic elements, such as MC Dropout
    targets = []
    predictions = []
    for X, y in dl:
        pred = model(X)
        targets += list(y.detach().cpu().numpy())
        predictions += list(pred.argmax(1).detach().cpu().numpy())
    accuracy = metrics.accuracy_score(targets, predictions)
    balanced_accuracy = metrics.balanced_accuracy_score(targets, predictions)
    print(f"Accuracy: {accuracy}, Balanced Accuracy: {balanced_accuracy}")

In [11]:
# Train model
learning_rate = 0.001
nr_epochs = 50
eval_every = 5
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_model(model, train_dl, optimizer, loss_f, nr_epochs, eval_every)

Epoch 0 loss 0.6181832194890616
Epoch 5 loss 0.522482619814153
Epoch 10 loss 0.41012479786602957
Epoch 15 loss 0.28879136163389907
Epoch 20 loss 0.21716988797492856
Epoch 25 loss 0.1701575539751365
Epoch 30 loss 0.14011782219519242
Epoch 35 loss 0.11939304139415535
Epoch 40 loss 0.10901243030670196
Epoch 45 loss 0.09452013022622988
Finished training!


In [12]:
# Evaluate model
print("Performance on the train set")
eval_model(model, train_dl)

print("Performance on the test set")
eval_model(model, test_dl)

Performance on the train set
Accuracy: 0.9810246679316889, Balanced Accuracy: 0.9804449521322889
Performance on the test set
Accuracy: 0.9261363636363636, Balanced Accuracy: 0.8774385072094996


## Restoring and saving models

### In order to make predictions in the future, we must save and restore model states

In [13]:
# Initialize new model, and evaluate
model = FullyConnectedClassifier(input_size=len(feature_cols), hidden_layer_sizes=[2, 2])
print("Performance on the train set")
eval_model(model, train_dl)

print("Performance on the test set")
eval_model(model, test_dl)

Performance on the train set
Accuracy: 0.3605313092979127, Balanced Accuracy: 0.2870430809399478
Performance on the test set
Accuracy: 0.375, Balanced Accuracy: 0.30296861747243425


In [14]:
# Now, let's restore our previous model state and evaluate anew
model.restore(state_name="last")

print("Performance on the train set")
eval_model(model, train_dl)

print("Performance on the test set")
eval_model(model, test_dl)

Performance on the train set
Accuracy: 0.9810246679316889, Balanced Accuracy: 0.9804449521322889
Performance on the test set
Accuracy: 0.9261363636363636, Balanced Accuracy: 0.8774385072094996


In [15]:
# Now let's build a larger model with 2 hidden layers
model_big = FullyConnectedClassifier(input_size=len(feature_cols), hidden_layer_sizes=[8, 16, 8])
print(model_big)

FullyConnectedClassifier(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax): Softmax(dim=1)
)


In [16]:
# We will train that model for the same number of epochs
optimizer = torch.optim.Adam(model_big.parameters(), lr=learning_rate)
train_model(model_big, train_dl, optimizer, loss_f, nr_epochs, eval_every)

print("Performance on the train set")
eval_model(model_big, train_dl)

print("Performance on the test set")
eval_model(model_big, test_dl)

# The mdoels are different, but are they significantly different?

Epoch 0 loss 0.6419557864373585
Epoch 5 loss 0.20466581948501286
Epoch 10 loss 0.12105143312000675
Epoch 15 loss 0.08235919920038351
Epoch 20 loss 0.05592728185962977
Epoch 25 loss 0.04390127951876728
Epoch 30 loss 0.028247982822702036
Epoch 35 loss 0.020187934383667016
Epoch 40 loss 0.015162830904238481
Epoch 45 loss 0.012960436522536263
Finished training!
Performance on the train set
Accuracy: 1.0, Balanced Accuracy: 1.0
Performance on the test set
Accuracy: 0.9829545454545454, Balanced Accuracy: 0.973960983884648


## *Null hypothesis*: the two ML models are equally accurate on the same cohort


## McNemar's Test

In [17]:
# Let's extract predictions from the two models
model.eval()
model_big.eval()
y_true = []
y_1 = []
y_2 = []
for X, y in test_dl:
    pred_1 = model(X)
    y_1 += list(pred_1.argmax(1).detach().cpu().numpy())
    pred_2 = model_big(X)
    y_2 += list(pred_2.argmax(1).detach().cpu().numpy())
    y_true += list(y.detach().cpu().numpy())

In [18]:
# Calculate contingency matrix
def contingency_table_calc(y_true, y_1, y_2):
    y_true, y_1, y_2 = np.array(y_true), np.array(y_1), np.array(y_2)
    c1i_c2c = sum(np.logical_and((y_1 != y_true),(y_2 == y_true)))
    c1c_c2i = sum(np.logical_and((y_1 == y_true),(y_2 != y_true)))
    c1c_c2c = sum(np.logical_and((y_1 == y_true),(y_2 == y_true)))
    c1i_c2i = sum(np.logical_and((y_1 != y_true),(y_2 != y_true)))

    contingency_table = [[c1c_c2c, c1c_c2i],[c1i_c2c, c1i_c2i]]

    return contingency_table

contingency_table = contingency_table_calc(y_true, y_1, y_2)

In [19]:
# Compute McNemar statistic and p-value to compare the two models
p = contingency_tables.mcnemar(contingency_table, exact=True).pvalue

print("Null Hypothesis: the two ML models are equally accurate on the same cohort.")
print(f'p-value with NcNemars exact test: {p:.4f}')
if p < 0.05:
    print("p<0.05 --> we reject the null hypothesis.")
else:
    print("p>0.05 --> we cannot reject the null hypothesis.")

Null Hypothesis: the two ML models are equally accurate on the same cohort.
p-value with NcNemars exact test: 0.0020
p<0.05 --> we reject the null hypothesis.
